## Compare NIS Importer Template with Country Template

### Set Up Requirements

In [10]:
pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Set up Macros and Packages

In [11]:
nis_download = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\sample data\DEM- Demography_20240826.xlsx"
country_download = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\sample data\DEM- Demography_20240829.xlsx"
summary_output= r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\testreport.txt"
additions_deletions = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\additions_deletions.xlsx"

import pandas as pd 
import datacompy

### Read in Recently Downloaded Importer Template

In [12]:
from_nis = pd.read_excel(io=f'{nis_download}', sheet_name ="Data", skiprows = 1)
from_nis.columns.values[0] = "Event UID"
from_nis.columns.values[1] = 'Organisation Unit*'
from_nis.columns.values[2] = 'Event Date'
from_nis.columns.values[3] = 'Latitude'
from_nis.columns.values[4] = 'Longitude'
from_nis.columns.values[5] = 'Status'
from_nis.drop(0, inplace=True)
print(from_nis)

         Event UID Organisation Unit*  Event Date Latitude Longitude  \
1      GMt7FtsC7wY              Danot  2012-10-01      NaN       NaN   
2      bZUPMoKcGFo              Danot  2013-10-01      NaN       NaN   
3      IKm73GEKmpi              Danot  2014-10-01      NaN       NaN   
4      OfhhArLODcI              Danot  2015-10-01      NaN       NaN   
5      gNoDqagN0c4              Danot  2016-10-01      NaN       NaN   
...            ...                ...         ...      ...       ...   
12127  lCGTVlyNkUt       Moyale Town   2020-10-01      NaN       NaN   
12128  goLVPnXrFDw       Moyale Town   2021-10-01      NaN       NaN   
12129  lauf58kHzsn       Moyale Town   2022-10-01      NaN       NaN   
12130  U7I0dwFWVni       Moyale Town   2023-10-01      NaN       NaN   
12131  cpkpf1tujfk       Moyale Town   2024-10-01      NaN       NaN   

          Status Review status   Country         Region Total population*  \
1      COMPLETED      Reviewed  Ethiopia            NaN   

c:\Users\ebradfield\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Generate Summary of Recently Downloaded Importer Template

In [13]:
from_nis.nunique(axis=0, dropna=True)

Event UID                                                           12131
Organisation Unit*                                                    937
Event Date                                                             13
Latitude                                                                0
Longitude                                                               0
Status                                                                  1
Review status                                                           2
Country                                                                 1
Region                                                                 84
Total population*                                                   11491
Source of population data                                               2
Other (please specify)                                                 11
Year of original population data collection                             1
Population growth rate (%)*           

### Read in Template from Country Team

In [15]:
from_country = pd.read_excel(io=f'{country_download}', sheet_name ="Data", skiprows = 1)
from_country.columns.values[0] = "Event UID"
from_country.columns.values[1] = 'Organisation Unit*'
from_country.columns.values[2] = 'Event Date'
from_country.columns.values[3] = 'Latitude'
from_country.columns.values[4] = 'Longitude'
from_country.columns.values[5] = 'Status'
from_country.drop(0, inplace=True)
print(from_country)

         Event UID Organisation Unit*           Event Date Latitude Longitude  \
1             test              Danot  2024-10-01 00:00:00      NaN       NaN   
2             test              Danot  2024-10-01 00:00:00      NaN       NaN   
3            test2              Danot  2024-10-01 00:00:00      NaN       NaN   
4            test3              Danot  2024-10-01 00:00:00      NaN       NaN   
5            test4              Danot  2024-10-01 00:00:00      NaN       NaN   
...            ...                ...                  ...      ...       ...   
12139  lCGTVlyNkUt       Moyale Town            2020-10-01      NaN       NaN   
12140  goLVPnXrFDw       Moyale Town            2021-10-01      NaN       NaN   
12141  lauf58kHzsn       Moyale Town            2022-10-01      NaN       NaN   
12142  U7I0dwFWVni       Moyale Town            2023-10-01      NaN       NaN   
12143  cpkpf1tujfk       Moyale Town            2024-10-01      NaN       NaN   

          Status Review sta

### Generate Summary of Template from Country Team

In [17]:
from_country.nunique(axis=0, dropna=True)

event uid                                                           12142
organisation unit*                                                    937
event date                                                             14
latitude                                                                0
longitude                                                               0
status                                                                  1
review status                                                           3
country                                                                 1
region                                                                 84
total population*                                                   11492
source of population data                                               2
other (please specify)                                                 11
year of original population data collection                             1
population growth rate (%)*           

### Merge Dataframes and Identify Discrepancies

In [16]:
compare = datacompy.Compare(from_nis, from_country, join_columns=['Event UID', 'Organisation Unit*'], df1_name='from_nis',
    df2_name='from_country')

print(compare.report(sample_count=1000000))

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

      DataFrame  Columns   Rows
0      from_nis       23  12131
1  from_country       23  12143

Column Summary
--------------

Number of columns in common: 23
Number of columns in from_nis but not in from_country: 0
Number of columns in from_country but not in from_nis: 0

Row Summary
-----------

Matched on: event uid, organisation unit*
Any duplicates on match values: Yes
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 12,131
Number of rows in from_nis but not in from_country: 0
Number of rows in from_country but not in from_nis: 12

Number of rows with some compared columns unequal: 11,221
Number of rows with all compared columns equal: 910

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 18
Total number of values which compare unequal: 11,225

Columns with Unequal Values or Types
--------

### Output Summary File

In [21]:
print(compare.all_mismatch())
with open(summary_output, 'w', encoding='utf-8') as report_file:
    report_file.write(compare.report(sample_count=1000000))


         event uid organisation unit* event date_from_nis  \
0      A09T9pEX5Lb       Endabatsahma          2015-10-01   
1      A0IynzQPo9P               Homa          2017-10-01   
2      A0jz2oqgVBY             Kelala          2017-10-01   
3      A120e2Kw0pL              Ejere          2012-10-01   
4      A1dhDvzJQB6       Hawasa Zuria          2015-10-01   
...            ...                ...                 ...   
12137  zy1EnX3Rpw6          Daba guna          2024-10-01   
12138  zy5HOYHM4Bl               Yalo          2016-10-01   
12139  zyqF3rA7fD4         Gida Ayana          2016-10-01   
12140  zzMFuHg3cYV      Kedida Gamela          2021-10-01   
12141  zzVFdkWcEyB             Boreda          2023-10-01   

      event date_from_country latitude_from_nis latitude_from_country  \
0                  2015-10-01               NaN                   NaN   
1                  2017-10-01               NaN                   NaN   
2                  2017-10-01               NaN 

### Write Additions and Deletions to Excel File

In [19]:
from_nis_new = compare.df1_unq_rows
from_country_new = compare.df2_unq_rows

from_nis_new.to_excel(additions_deletions, sheet_name="From NIS", index = False)

with pd.ExcelWriter(additions_deletions, mode="a", engine="openpyxl") as writer:
    from_country_new.to_excel(writer, sheet_name = "From Country", index = False)